# Implementing tile overlap

The example that this process is based off is only 2D image. 

I have a 4D stack so I need to separate each layer and tile individually.

Now tiling over time and z stack, but testing to see if order of iteration matters. 

In [1]:
import napari
import matplotlib.pyplot as plt
import glob
import os
import octopusheavy as octo
import pandas as pd
from tqdm.auto import tqdm
from functools import partial
from octopusheavy import tile
from skimage.io import imread
import dask.array as da

#### Find images

In [2]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
# image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/r03c03f01p01/'
fns = glob.glob(os.path.join(image_dir, '*.tiff'))
# fns = glob.glob(os.path.join(image_dir, f'r0{row}c0{col}*.tiff'))
print(len(fns), 'image files found')

138402 image files found


### Loading metadata

In [3]:
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
df = octo.utils.read_harmony_metadata(metadata_fn)

Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

#### Get position information from metadata

In [4]:
row_col_list = list()
for index, row in tqdm(df.iterrows(), total = len(df)):
    row_col_list.append(tuple((int(row['Row']), int(row['Col']))))
row_col_list = list(set(row_col_list))
for n, i in enumerate(row_col_list):
    print('Position index and (row,column):', n, i)

  0%|          | 0/113400 [00:00<?, ?it/s]

Position index and (row,column): 0 (3, 4)
Position index and (row,column): 1 (4, 3)
Position index and (row,column): 2 (4, 9)
Position index and (row,column): 3 (3, 7)
Position index and (row,column): 4 (5, 4)
Position index and (row,column): 5 (4, 6)
Position index and (row,column): 6 (3, 10)
Position index and (row,column): 7 (5, 7)
Position index and (row,column): 8 (6, 5)
Position index and (row,column): 9 (6, 8)
Position index and (row,column): 10 (4, 5)
Position index and (row,column): 11 (3, 3)
Position index and (row,column): 12 (3, 9)
Position index and (row,column): 13 (5, 6)
Position index and (row,column): 14 (4, 8)
Position index and (row,column): 15 (3, 6)
Position index and (row,column): 16 (5, 9)
Position index and (row,column): 17 (6, 4)
Position index and (row,column): 18 (6, 7)
Position index and (row,column): 19 (4, 7)
Position index and (row,column): 20 (3, 5)
Position index and (row,column): 21 (4, 4)
Position index and (row,column): 22 (4, 10)
Position index and 

## Get dimensionality of image volume

In [5]:
channel_IDs = df['ChannelID'].unique()
plane_IDs = df['PlaneID'].unique()
timepoint_IDs = df['TimepointID'].unique()

## Set mosaic parameters

The `chunk_fraction` is how many sections you want one slice cut up into (has to be a square) and the `_load_image` partial function can include any image transformations you wish (border crop or background removal etc).

In [6]:
chunk_fraction = 49
load_transform_image = partial(tile.load_image, transforms=[])#input_transforms)

In [7]:
# _fuse_func=partial(tile.fuse_func, 
#                    imload_fn=load_transform_image,
#                    dtype=imread(fns[0]).dtype)

## Set segmentation parameters

In [8]:
!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

model = models.Cellpose(gpu=True, model_type='cyto')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Fri Dec 16 11:41:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:65:00.0  On |                  Off |
| 37%   62C    P2    86W / 300W |  13410MiB / 49140MiB |      3%      Default |
|                               |                      |                  N/A |
+

In [17]:
def segment(img):
    masks, flows, styles, diams = model.eval(img, diameter=200, channels=[0,0],
                                             flow_threshold=None, cellprob_threshold=0)
    return masks

In [13]:
glob.glob('/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/*.tiff')[0]

'/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/r03c06f08p02-ch1sk36fk1fl1.tiff'

In [ ]:
im

In [18]:
for i in tqdm(range(1000)):
    img = imread(glob.glob('/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/*.tiff')[0])
    segment(img)

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [19]:
from skimage.transform import rescale, resize

In [21]:
img.shape

(2160, 2160)

(21600, 21600)

In [25]:
resize(img, (img.shape[0]*10, img.shape[0]*10))

array([[0.00188251, 0.00187701, 0.00187152, ..., 0.00178855, 0.00179747,
        0.0018064 ],
       [0.00188464, 0.00188037, 0.0018761 , ..., 0.00179885, 0.00180884,
        0.00181884],
       [0.00188678, 0.00188373, 0.00188067, ..., 0.00180915, 0.00182021,
        0.00183127],
       ...,
       [0.01425803, 0.01433127, 0.01440452, ..., 0.00619135, 0.00617456,
        0.00615778],
       [0.01431662, 0.01438499, 0.01445335, ..., 0.00619745, 0.00617945,
        0.00616144],
       [0.01437522, 0.0144387 , 0.01450217, ..., 0.00620356, 0.00618433,
        0.0061651 ]])

In [26]:
for i in tqdm(range(1000)):
    img = imread(glob.glob('/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/*.tiff')[0])
    img = resize(img, (img.shape[0]*10, img.shape[0]*10))
    segment(img)

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Compile and segment at the same time...? 

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
z_mask_stack = []
t_mask_stack = []
images = dict()
masks = dict()
for row, col in tqdm(row_col_list, desc = 'Position progress'):
    ### define row and col
    row, col = str(row), str(col)
    ### clear empty arrays for organsing into dask arrays
    t_stack = []
    t_mask_stack = []
    ### iterate over each individual image slice, tiling together
    for time in tqdm(timepoint_IDs, leave = False, desc = 'Timepoint progress'):
        c_stack = []
        for channel in tqdm(channel_IDs, leave = False, desc = 'Channel progress'):
            z_stack = []
            if channel == '1':
                z_mask_stack = []
            for plane in tqdm(plane_IDs, leave = False, desc = 'Z-slice progress'):
                frame = tile.stitch(load_transform_image,
                                    df, 
                                    image_dir, 
                                    time, 
                                    plane, 
                                    channel, 
                                    row, 
                                    col, 
                                    chunk_fraction)
                ### if we're iterating over the GFP channel then segment that
                if channel == '1':
                    masks_frame = segment(frame)
                    z_mask_stack.append(masks_frame)
                ### collect stitched frames together into time stack
                z_stack.append(frame)
            ### stack channel series together (images)
            c_stack.append(z_stack)
        ### stack together timewise
        t_stack.append(c_stack)
        t_mask_stack.append(z_mask_stack)
    ### stack stitched dask arrays together into multidim image volumes
    images[(int(row), int(col))] = da.stack([da.stack(c_stack, axis = 0) for c_stack in t_stack])
    masks[(int(row), int(col))] = da.stack(t_mask_stack)

Position progress:   0%|          | 0/28 [00:00<?, ?it/s]

Timepoint progress:   0%|          | 0/75 [00:00<?, ?it/s]

Channel progress:   0%|          | 0/2 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

...

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Channel progress:   0%|          | 0/2 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

NameError: name 'da' is not defined

In [18]:
images[(3, 4)]

KeyError: (3, 4)

In [ ]:
stop

# Check segment

In [ ]:
v = napari.Viewer()
v.add_image(images[(3, 4)],
            channel_axis=1,
            name=["macrophage", "mtb"],
            colormap=["green", "magenta"],
            )  
# v.add_image(images[(3, 4)])
v.add_labels(masks[(3, 4)])